# Personal attacks in comments

In [24]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",250)
pd.set_option("display.max_columns",250)
import unicodedata
import chardet
import re
import string
import nltk
from nltk.corpus import stopwords
import numpy as np
import os
from nltk.tokenize import regexp_tokenize
from nltk.stem.porter import PorterStemmer
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from  nltk.corpus import opinion_lexicon
from nltk.stem.snowball import EnglishStemmer
english_stem = EnglishStemmer(ignore_stopwords=True)


In [25]:
dataset = pd.read_csv("attack_annotations.tsv", delimiter="\t")
dataset.sum()

rev_id                4.095303e+14
worker_id             1.828416e+09
quoting_attack        1.027300e+04
recipient_attack      1.526070e+05
third_party_attack    4.457100e+04
other_attack          4.412100e+04
attack                2.279360e+05
dtype: float64

In [26]:
revisions = pd.read_csv("attack_annotated_comments.tsv", delimiter="\t")

In [27]:
# Identifier les commentaires faisant l'objet d'une attaque
rev_attacks = []
for i in range(len(dataset)) :
    if dataset["attack"][i] == 1 :
        rev_attacks.append(dataset["rev_id"][i])
rev_attacks = list(set(rev_attacks))

In [28]:
# Ajout d'une colonne "attacks"
revisions["attacks"] = revisions["rev_id"].isin(rev_attacks) 

In [29]:
revisions.head()

,rev_id,comment,year,logged_in,ns,sample,split,attacks
0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train,False
1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train,False
2,49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",2002,False,article,random,train,False
3,89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev,True
4,93890,This page will need disambiguation.,2002,True,article,random,train,False


In [52]:
# Jeu d'entrainement
revisions_train = revisions[revisions["split"]== "train"][:100000]

In [53]:
revisions_train.shape

(69526, 8)

## Ces revisions ont-elles fait l'objet d'une attaque ? 

In [54]:
comments = revisions_train["comment"].tolist()
print comments[24]

NEWLINE_TOKENNEWLINE_TOKENHi AlexNEWLINE_TOKENNEWLINE_TOKENJust thought I ought to mention that not ALL self-links are unwanted - you removed one in List of musical topics that was wanted and its presence was requested on the Talk page. I think there are a few in maths topics which make this clear - something to do with keeping an eye on updates? It's not clear to me how this whole issue works (in fact I thought I'd raise it in the village pump) some time, but thought I should mention it to you anway. Regards  11:33 Dec 26, 2002 (UTC)


In [55]:
# Decode les commentaires
commentaires = []
for comment in comments :
    encoding =  chardet.detect(comment)
    comment = unicodedata.normalize("NFKD", comment.decode(encoding["encoding"])).encode("ASCII", 'ignore').lower().decode("ASCII")
    commentaires.append(comment)

In [56]:


listComments = []
for comment in commentaires :
    # remplacer toute la ponctuation par des espaces
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    # découpe la string à chaque espace
    elements = regex.sub(' ', comment).split()
    # applique la racination 
    stemed_elements = [english_stem.stem(x) for x in elements if len(x)>2]
    listComments.append(stemed_elements)
    
listComments = [' '.join(mylist) for mylist in listComments]

### 1er Corpus : SentimentIntensityAnalyzer

In [57]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

# True si positif ou neutre, False si negatif
sentiment = []
for sentence in listComments :
    ss = sid.polarity_scores(sentence)
    sentiment.append(ss['compound']>=0)
    
revisions_train["sentiment"] = sentiment   

In [58]:
revisions_train.head()

,rev_id,comment,year,logged_in,ns,sample,split,attacks,sentiment
0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train,False,False
1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train,False,True
2,49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",2002,False,article,random,train,False,True
4,93890,This page will need disambiguation.,2002,True,article,random,train,False,True
5,102817,NEWLINE_TOKEN-NEWLINE_TOKENNEWLINE_TOKENImport...,2002,True,user,random,train,False,True


In [59]:
accuracy = np.mean(revisions_train["sentiment"] != revisions_train["attacks"])
print accuracy

0.641285274574


## 2nd Corpus : opinion_lexicon

In [60]:
# Corpus des termes d'opinions négatives

negatifs = []

for words in opinion_lexicon.negative() :
    # stems 
    stemed_elements = [english_stem.stem(words)]
    negatifs.extend(stemed_elements)
negatifs = list(set(negatifs))

print negatifs[:10]

[u'broken-heart', u'fawn', u'lawbreak', u'hatr', u'foul', u'protest', u'thirst', u'tenuous', u'hate', u'inexpert']


In [61]:
# Stemmers
from nltk.stem.porter import *

stemmer = PorterStemmer()
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token) and not re.search("''", token) and not re.search('[0-9]', token) and len(token)>3:
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
            
    stems = [word for word in stems if word in negatifs]
    return stems

In [62]:
# stemmer = Porter 
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem, decode_error='ignore',  max_df=0.95, min_df=2)
tfidf = tfidf_vectorizer.fit_transform(listComments)

In [63]:
%time tfidf_matrix = tfidf_vectorizer.fit_transform(listComments)

indices = np.argsort(tfidf_vectorizer.idf_)[::-1]
features = tfidf_vectorizer.get_feature_names()
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
print top_features

Wall time: 6min 44s
[u'griev', u'clamor', u'caustic', u'grisli', u'chaff', u'odor', u'cheapli', u'graini', u'glib', u'giddi', u'gibber', u'ghast', u'garish', u'overact', u'overdon', u'treacher', u'overplay', u'tetchi', u'testi', u'gaff']


In [64]:
tfidf_matrix.shape

(69526, 2015)

In [65]:
X2 = tfidf_matrix.toarray()
Y2 = revisions_train["attacks"].as_matrix()

In [66]:
senti = np.asarray(revisions_train["sentiment"].tolist())
my = senti[:, np.newaxis]

In [67]:
X2 = np.append(X2, my, 1)
X2.shape

(69526L, 2016L)

In [68]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.linear_model import Perceptron as PPN
from sklearn.metrics import average_precision_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, f1_score

def run_cv(X,y,clf_class,**kwargs):
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.20)
    y_pred = y.copy()
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
        accuracy = np.mean(y_test == y_pred[test_index])
    return accuracy

print "Random forest:"
print "%.3f" % run_cv(X2,Y2,RF)

Random forest:
0.678
